In [1]:
import pandas as pd
from transformers import AutoTokenizer, TFAutoModelForQuestionAnswering, pipeline, TFAutoModelForSequenceClassification
import os
import re
import time
import logging

In [2]:
BASE_DIR = '/home/u132668/'

In [3]:
for name in logging.root.manager.loggerDict:
        if re.match("transformers", name):
            logging.getLogger(name).setLevel(logging.ERROR)

In [4]:
test = pd.read_csv(os.path.join(BASE_DIR, 'data/test.csv'))

In [16]:
verbs = set(['am','is','are','was','were','has','have','had','shall','will','can','should','would','could','must','may','might','do','does','did'])

In [17]:
def check_boolq(question):
    first_word = question.split(' ')[0]
    if first_word in verbs:
        return 1
    if question.rfind('true or false')>=0:
        return 2
    return 0

In [18]:
test['Question'] = test['Question'].apply(lambda x: x.lower().strip())
test['check_boolq'] = test['Question'].apply(check_boolq)

In [5]:
tokenizer_qa = AutoTokenizer.from_pretrained('WaRKiD/bert-large-uncased-whole-word-masking-finetuned-intel-oneapi-llm-dataset')
model_qa = TFAutoModelForQuestionAnswering.from_pretrained('WaRKiD/bert-large-uncased-whole-word-masking-finetuned-intel-oneapi-llm-dataset')

In [6]:
model_qa_pipeline = pipeline("question-answering", model=model_qa, tokenizer=tokenizer_qa, device=0)

In [7]:
start_start = time.time()
result = model_qa_pipeline(question=test.iloc[:]['Question'].to_list(), context=test.iloc[:]['Story'].to_list())
end_time = time.time()

In [8]:
print("The time of execution of above program is :", (end_time-start_start) * 10**3,"ms")

The time of execution of above program is : 7145624.570131302 ms


In [9]:
answers = []
for i in range(len(result)):
    answers.append(result[i]['answer'])

In [10]:
submission = pd.read_csv('D:\\ML\\oneAPI Hackathon - The LLM Challenge\\submission.csv')

In [13]:
submission['Answer'] = answers

In [34]:
submission.to_csv('D:\\ML\\oneAPI Hackathon - The LLM Challenge\\out\\submission_v9.csv',index=False)

In [20]:
tokenizer_tf = AutoTokenizer.from_pretrained('WaRKiD/distilbert-base-uncased-finetuned-intel-llm-tf-dataset')
model_tf = TFAutoModelForSequenceClassification.from_pretrained('WaRKiD/distilbert-base-uncased-finetuned-intel-llm-tf-dataset')

In [23]:
model_tf_pipeline = pipeline("text-classification", model=model_tf, tokenizer=tokenizer_tf, device=0)

In [25]:
result_tf = model_tf_pipeline((test[test['check_boolq'] == 2]['Question']+ ' ' +submission.iloc[test[test['check_boolq'] == 2].index]['Answer']).to_list())

In [26]:
for i,j in enumerate(test[test['check_boolq'] == 2].index):
    submission.iloc[j]['Answer'] = result_tf[i]['label']

In [28]:
tokenizer_yn = AutoTokenizer.from_pretrained('WaRKiD/distilbert-base-uncased-finetuned-intel-llm-yn-dataset')
model_yn = TFAutoModelForSequenceClassification.from_pretrained('WaRKiD/distilbert-base-uncased-finetuned-intel-llm-yn-dataset')

In [30]:
model_yn_pipeline = pipeline("text-classification", model=model_yn, tokenizer=tokenizer_yn, device=0)

In [31]:
result_yn = model_yn_pipeline((test[test['check_boolq'] == 1]['Question']+ ' ' +submission.iloc[test[test['check_boolq'] == 1].index]['Answer']).to_list())

In [33]:
for i,j in enumerate(test[test['check_boolq'] == 1].index):
    submission.iloc[j]['Answer'] = result_yn[i]['label']